## Bulk2Space Tutorial

### Outline

In [1]:
from utils.tool import *
from utils.config import cfg, loadArgums
import numpy as np
import pandas as pd
import torch
import scanpy
from scipy.optimize import nnls
from collections import defaultdict
import argparse
import warnings
warnings.filterwarnings('ignore')

In [2]:
global args 
args = dict(
    BetaVAE_H=True,
    batch_size=512,
    cell_num=10,
    data_path='example_data/demo1',
    dump_path='/data/zhuangxiang/code/bulk2space/bulk2space/dump',
    early_stop=50,
    epoch_num=10,
    exp_id='LR_0.0001_hiddenSize_256_lay_choice_0',
    exp_name='test1',
    feature_size=6588,
    gpu_id=0,
    hidden_lay=0,
    hidden_size=256,
    input_bulk_path='/data/zhuangxiang/code/bulk2space/bulk2space/data/example_data/demo1/demo1_bulk.csv',
    input_sc_data_path='/data/zhuangxiang/code/bulk2space/bulk2space/data/example_data/demo1/demo1_sc_data.csv',
    input_sc_meta_path='/data/zhuangxiang/code/bulk2space/bulk2space/data/example_data/demo1/demo1_sc_meta.csv',
    input_st_data_path='/data/zhuangxiang/code/bulk2space/bulk2space/data/example_data/demo1/demo1_st_data.csv',
    input_st_meta_path='/data/zhuangxiang/code/bulk2space/bulk2space/data/example_data/demo1/demo1_st_meta.csv',
    k=10, kl_loss=False, learning_rate=0.0001, load_model_1=False, load_path_1='/data/zhuangxiang/code/bulk2space/bulk2space/save_model/',
    load_path_2='/data/zhuangxiang/code/bulk2space/bulk2space/save_model/', marker_used=True, max_cell_in_diff_spot_ratio=None, model_choice_1='vae', model_choice_2='df',
    mul_test=5, mul_train=1, no_tensorboard=False, not_early_stop=False, num_workers=12, output_path='/data/zhuangxiang/code/bulk2space/bulk2space/output_data',
    previous_project_name='demo', project_name='test1', random_seed=12345, ratio_num=1,
    save='/data/zhuangxiang/code/bulk2space/bulk2space/save_model', spot_data=True, spot_num=500,
    top_marker_num=500, train_model_2=True, xtest='xtest', xtrain='xtrain', ytest='ytest', ytrain='ytrain'
)
args = argparse.Namespace(**args)

In [3]:
input_sc_meta_path = args.input_sc_meta_path
input_sc_data_path = args.input_sc_data_path
input_bulk_path = args.input_bulk_path
input_st_meta_path = args.input_st_meta_path
input_st_data_path = args.input_st_data_path
print("loading data......")

# load sc_meta.csv file, containing two columns of cell name and cell type
input_sc_meta = pd.read_csv(input_sc_meta_path, index_col=0)
# load sc_data.csv file, containing gene expression of each cell
input_sc_data = pd.read_csv(input_sc_data_path, index_col=0)
sc_gene = input_sc_data._stat_axis.values.tolist()
# load bulk.csv file, containing one column of gene expression in bulk
input_bulk = pd.read_csv(input_bulk_path, index_col=0)
bulk_gene = input_bulk._stat_axis.values.tolist()
# filter overlapping genes.
intersect_gene = list(set(sc_gene).intersection(set(bulk_gene)))
input_sc_data = input_sc_data.loc[intersect_gene]
input_bulk = input_bulk.loc[intersect_gene]
# load st_meta.csv and st_data.csv, containing coordinates and gene expression of each spot respectively.
input_st_meta = pd.read_csv(input_st_meta_path, index_col=0)
input_st_data = pd.read_csv(input_st_data_path, index_col=0)
print("load data ok")

loading data......
load data ok


In [4]:
sc = scanpy.AnnData(input_sc_data.T)
sc.obs = input_sc_meta[['Cell_type']]
scanpy.tl.rank_genes_groups(sc, 'Cell_type', method='wilcoxon')
marker_df = pd.DataFrame(sc.uns['rank_genes_groups']['names']).head(args.top_marker_num)
marker_array = np.array(marker_df)
marker_array = np.ravel(marker_array)
marker_array = np.unique(marker_array)
marker = list(marker_array)
sc_marker = input_sc_data.loc[marker, :]
bulk_marker = input_bulk.loc[marker]


In [5]:
breed = input_sc_meta['Cell_type']
breed_np = breed.values
breed_set = set(breed_np)
id2label = sorted(list(breed_set))  # List of breed
label2id = {label: idx for idx, label in enumerate(id2label)}  # map breed to breed-id

cell2label = dict()  # map cell-name to breed-id
label2cell = defaultdict(set)  # map breed-id to cell-names
for row in input_sc_meta.itertuples():
    cell_name = getattr(row, 'Cell')
    cell_type = label2id[getattr(row, 'Cell_type')]
    cell2label[cell_name] = cell_type
    label2cell[cell_type].add(cell_name)

In [6]:
label_devide_data = dict()
for label, cells in label2cell.items():
    label_devide_data[label] = sc_marker[list(cells)]

single_cell_splitby_breed_np = {}
for key in label_devide_data.keys():
    single_cell_splitby_breed_np[key] = label_devide_data[key].values  # [gene_num, cell_num]
    single_cell_splitby_breed_np[key] = single_cell_splitby_breed_np[key].mean(axis=1)

max_decade = len(single_cell_splitby_breed_np.keys())
single_cell_matrix = []
#
for i in range(max_decade):
    single_cell_matrix.append(single_cell_splitby_breed_np[i].tolist())


single_cell_matrix = np.array(single_cell_matrix)
single_cell_matrix = np.transpose(single_cell_matrix)  # (gene_num, label_num)

bulk_marker = bulk_marker.values  # (gene_num, 1)
bulk_rep = bulk_marker.reshape(bulk_marker.shape[0],)

# calculate celltype ratio in each spot by NNLS
ratio = nnls(single_cell_matrix, bulk_rep)[0]
ratio = ratio/sum(ratio)

ratio_array = np.round(ratio * input_sc_meta.shape[0] * args.ratio_num)
ratio_list = [r for r in ratio_array]

# pdb.set_trace()
cell_target_num = dict(zip(id2label, ratio_list))

# *********************************************************************
# input：data， celltype， bulk & output: label, dic, single_cell
single_cell = input_sc_data.values.T  # single cell data (600 * 6588)
index_2_gene = (input_sc_data.index).tolist()
breed = input_sc_meta['Cell_type']
breed_np = breed.values
breed_set = set(breed_np)
breed_2_list = list(breed_set)
dic = {}  # breed_set to index {'B cell': 0, 'Monocyte': 1, 'Dendritic cell': 2, 'T cell': 3}
label = []  # the label of cell (with index correspond)
cfg.nclass = len(breed_set)

cfg.ntrain = single_cell.shape[0]
cfg.FeaSize = single_cell.shape[1]
args.feature_size = single_cell.shape[1]
assert cfg.nclass == len(cell_target_num.keys()), "cell type num no match!!!"

for i in range(len(breed_set)):
    dic[breed_2_list[i]] = i
cell = input_sc_meta["Cell"].values

for i in range(cell.shape[0]):
    label.append(dic[breed_np[i]])

label = np.array(label)

# label index the data size of corresponding target
cell_number_target_num = {}
for k, v in cell_target_num.items():
    cell_number_target_num[dic[k]] = v
# pdb.set_trace()
# *********************************************************************
# generate data by vae
load_model_1 = args.load_model_1
model_choice_1 = args.model_choice_1

In [7]:
ratio = -1
if not load_model_1:  # train
    print("begin vae model training...")
    # ********************* training *********************
    net = train_vae(args, single_cell, cfg, label)
    # ************** training finished *******************
    print("vae training finished!")
else:  # load model
    print("begin vae model loading...")
    net = load_vae(args, cfg)
    print("vae load finished!")


begin vae model training...


Train Epoch: 9: 100%|██████████| 10/10 [00:09<00:00,  1.01it/s, loss=0.7232, min_loss=0.7232]


min loss = 0.7232240438461304
vae training finished!


In [8]:
# generate and out put
generate_sc_meta, generate_sc_data = generate_vae(net, args, ratio, single_cell, cfg, label, breed_2_list, index_2_gene, cell_number_target_num)


# saving.....
path = osp.join(args.output_path, args.project_name, 'predata')
if not osp.exists(path):
    os.makedirs(path)
name = "vae"
# kl_loss BetaVAE_H
if args.BetaVAE_H:
    name = "BetaVAE"
path_label_generate_csv = os.path.join(path, args.project_name + "_celltype_pred_" + name + "_epoch" + str(args.epoch_num) + '_lr' + str(args.learning_rate) + ".csv")
path_cell_generate_csv = os.path.join(path, args.project_name + "_data_pred_" + name + "_epoch" + str(args.epoch_num) + '_lr' + str(args.learning_rate) + ".csv")

generate_sc_meta.to_csv(path_label_generate_csv)
generate_sc_data.to_csv(path_cell_generate_csv)

print("bulk deconvolution finish!")

print('start to map data to space...')

Generate Epoch: 3: 100%|██████████| 249/249.0 [00:02<00:00, 97.82it/s] 


generated done!
begin data to spatial mapping...
Data have been prepared...
bulk deconvolution finish!
start to map data to space...


In [9]:
generate_sc_meta

,Cell,Cell_type
0,C_1,Monocyte
1,C_2,Monocyte
2,C_3,Monocyte
3,C_4,Dendritic cell
4,C_5,T cell
...,...,...
244,C_245,Dendritic cell
245,C_246,Dendritic cell
246,C_247,Dendritic cell
247,C_248,Dendritic cell


In [10]:
generate_sc_data

,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10,...,C_240,C_241,C_242,C_243,C_244,C_245,C_246,C_247,C_248,C_249
LSM7,0.169653,0.166711,0.108118,0.110623,0.225975,0.112983,0.127049,0.084420,0.199797,0.257178,...,0.267260,0.154195,0.194103,0.209751,0.162487,0.116674,0.146005,0.156942,0.119829,0.203258
PPARA,0.000000,0.000000,0.008387,0.059203,0.000000,0.019428,0.000000,0.000000,0.000000,0.061828,...,0.016695,0.027112,0.000000,0.023074,0.006484,0.026559,0.000000,0.021501,0.011669,0.000000
SLFN12,0.056475,0.014652,0.065971,0.018436,0.076505,0.048508,0.042563,0.000000,0.000000,0.016734,...,0.126016,0.013162,0.040907,0.068904,0.008105,0.006622,0.001837,0.062377,0.057931,0.021447
GGCX,0.000000,0.022952,0.000000,0.000000,0.000000,0.043131,0.000000,0.000000,0.000000,0.004719,...,0.001617,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032767,0.000000,0.000000
ATP6V1G1,0.216032,0.284197,0.253880,0.339176,0.236737,0.248473,0.256555,0.198254,0.236151,0.223107,...,0.356671,0.231773,0.306707,0.253625,0.217933,0.319698,0.293528,0.278796,0.235175,0.202648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
APP,0.140931,0.157144,0.099045,0.098334,0.118056,0.088388,0.123823,0.117236,0.129853,0.055534,...,0.182692,0.123914,0.136280,0.093900,0.094613,0.129129,0.112704,0.151161,0.146701,0.110071
COMT,0.003974,0.014249,0.021470,0.079107,0.000000,0.016548,0.056870,0.081998,0.071208,0.035738,...,0.039838,0.032895,0.053768,0.024900,0.003929,0.080014,0.049663,0.062463,0.033948,0.000000
ZC3H7B,0.068797,0.031826,0.051612,0.051913,0.042589,0.036428,0.146827,0.079889,0.095069,0.060124,...,0.105718,0.053005,0.061713,0.093156,0.004960,0.083887,0.121561,0.060474,0.003652,0.044843
MAP3K8,0.112597,0.105967,0.114579,0.099765,0.118407,0.178766,0.092427,0.087266,0.106486,0.091979,...,0.224640,0.116123,0.114401,0.128218,0.116354,0.100272,0.105666,0.148119,0.130832,0.122209


### Mapping generated single cells to spatial locations
After obtaining the generated data, we needed to map each cell and the spot it belongs to.

In [11]:
processer = CreatData(generate_sc_data, generate_sc_meta, input_st_data, args)
processer.cre_data()
runner = Runner(generate_sc_data, generate_sc_meta, input_st_data, input_st_meta, args)
print('start to train the model...')
pred_meta, pred_spot = runner.run()
print("spatial mapping done!")

preparing train data...
sucessfully create positive data
sucessfully create negative data
save xtrain ok！
save ytrain ok！
train data already prepared.
load xtrain ok!
load ytrain ok!
select top 500 marker genes of each cell type...
start to train the model...
saving model done!
model trained sucessfully, start saving output ...
Calculating scores...
Calculating scores done.
save csv ok
spatial mapping done!


Predicted results, including cell, cell type, spot to which cell belongs, spot coordinates, cell coordinates.

In [12]:
pred_meta

,Cell,Cell_type,Spot,Spot_xcoord,Spot_ycoord,Cell_xcoord,Cell_ycoord
0,C_1,B cell,spot_1,1,1,1.03,1.07
1,C_2,T cell,spot_1,1,1,1.08,1.09
2,C_3,B cell,spot_1,1,1,1.22,0.83
3,C_4,T cell,spot_1,1,1,0.68,1.03
4,C_5,B cell,spot_1,1,1,1.08,0.74
...,...,...,...,...,...,...,...
295,C_296,B cell,spot_30,6,5,6.26,5.02
296,C_297,B cell,spot_30,6,5,5.92,5.33
297,C_298,Dendritic cell,spot_30,6,5,6.21,4.94
298,C_299,Dendritic cell,spot_30,6,5,6.21,4.74


Predicted results, cell-gene expression matrix.

In [13]:
pred_spot

,C_1,C_2,C_3,C_4,C_5,C_6,C_7,C_8,C_9,C_10,...,C_291,C_292,C_293,C_294,C_295,C_296,C_297,C_298,C_299,C_300
LSM7,0.133734,0.195355,0.179828,0.141191,0.203839,0.167809,0.171385,0.218784,0.081844,0.179431,...,0.137828,0.195355,0.141191,0.207271,0.182715,0.203839,0.167809,0.146005,0.183277,0.188099
PPARA,0.000000,0.018602,0.000000,0.000000,0.005308,0.000000,0.065749,0.044647,0.000000,0.081925,...,0.009744,0.018602,0.000000,0.000000,0.000000,0.005308,0.000000,0.000000,0.018919,0.010499
SLFN12,0.064737,0.062833,0.054164,0.014783,0.112862,0.023568,0.041597,0.023094,0.045225,0.087671,...,0.044654,0.062833,0.014783,0.071534,0.000000,0.112862,0.023568,0.001837,0.048997,0.042528
GGCX,0.001422,0.000000,0.000000,0.000000,0.015827,0.000000,0.000000,0.001247,0.000000,0.037379,...,0.000000,0.000000,0.000000,0.000000,0.012674,0.015827,0.000000,0.000000,0.016768,0.000000
ATP6V1G1,0.233016,0.237005,0.267046,0.275500,0.234424,0.213815,0.245357,0.239580,0.267036,0.346584,...,0.265613,0.237005,0.275500,0.218831,0.217896,0.234424,0.213815,0.293528,0.280963,0.292195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
APP,0.130617,0.139232,0.147832,0.072008,0.056766,0.119591,0.085077,0.094390,0.093887,0.143359,...,0.103816,0.139232,0.072008,0.193768,0.129179,0.056766,0.119591,0.112704,0.107210,0.073020
COMT,0.002355,0.049854,0.049619,0.011113,0.064505,0.074899,0.060457,0.038896,0.061960,0.060151,...,0.021719,0.049854,0.011113,0.056933,0.054570,0.064505,0.074899,0.049663,0.045906,0.094033
ZC3H7B,0.039649,0.088281,0.054741,0.031150,0.035807,0.036274,0.135929,0.015703,0.029376,0.085057,...,0.093456,0.088281,0.031150,0.058778,0.011365,0.035807,0.036274,0.121561,0.048165,0.058367
MAP3K8,0.123249,0.223986,0.079975,0.132165,0.100193,0.107975,0.078321,0.091642,0.141444,0.175359,...,0.133929,0.223986,0.132165,0.175625,0.184559,0.100193,0.107975,0.105666,0.108682,0.113194
